In [1]:
from django.http.response import Http404, HttpResponseRedirect
from django.urls import reverse
from django.shortcuts import render, get_list_or_404, get_object_or_404
from django.views.decorators.clickjacking import xframe_options_sameorigin
from django.http import HttpResponse

import os, sys
os.chdir('/Users/wxy/OneDrive/projects/website/gathery')
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from user.models import User
from gallery.models import Gallery,Warehouse,Artist,Art,GU_Relation,WA_Collection,GW_Attach,Test_relation2,Test_relation1
from django.utils import timezone
import datetime
import json,random,string




NameError: name '_mysql' is not defined

In [2]:
w_ids = list(Warehouse.objects.all().values_list('id',flat=True))
g_ids = list(Gallery.objects.all().values_list('id',flat=True))
u_ids = list(User.objects.all().values_list('id',flat=True))

In [3]:
#get random user
def pickUserId(a,b):
    min_u = a
    max_u = b
    u_n = random.randint(min_u, max_u)
    rr = random.sample(u_ids, u_n)
    return User.objects.filter(id__in = rr)

def pickGalleryId(a,b):
    min_u = a
    max_u = b
    u_n = random.randint(min_u, max_u)
    rr = random.sample(g_ids, u_n)
    return Gallery.objects.filter(id__in = rr)

def pickWarehouseId(a,b):
    min_u = a
    max_u = b
    u_n = random.randint(min_u, max_u)
    rr = random.sample(w_ids, u_n)
    return Warehouse.objects.filter(id__in = rr)




R1 + R2 / W /   
13.5 s ± 113 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)  
14.3 s ± 816 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)  



In [4]:
users = pickUserId(100,200)
gs = pickGalleryId(70,80)
ws = pickWarehouseId(700,750)

In [5]:
users = pickUserId(10,11)

In [32]:
len(users)*len(ws)

7050

In [6]:
# 0.02024904214559387
# R2 → R1 / W / 
#

In [29]:
# %%timeit -n 1
s = len(users)*len(ws)
v = 0
for u in users:
    for w in ws:
        wg = w.gallery
        if Test_relation2.objects.filter(staff_member = u, warehouse = w).exists():
            v = v+1
        elif Test_relation1.objects.filter(staff_member = u, gallery = wg).exists():
            v = v+1
print(v/s)


0.019148936170212766


In [8]:
#
# R1 → R2 / W / 
#

In [30]:
# %%timeit -n 1
s = len(users)*len(ws)
v = 0
for u in users:
    for w in ws:
        wg = w.gallery
        if Test_relation1.objects.filter(staff_member = u, gallery = wg).exists():
            v = v+1
        elif Test_relation2.objects.filter(staff_member = u, warehouse = w).exists():
            v = v+1
print(v/s)


0.019148936170212766


In [10]:
#
# GU / W /
# 


In [31]:
#%%timeit -n 1
from django.db.models import Q
s = len(users)*len(ws)
v = 0
for u in users:
    for w in ws:
        wg = w.gallery
        if GU_Relation.objects.filter( Q(gallery = wg, warehouse__isnull = True) | Q(warehouse = w),
            staff_member = u
        ).exists():
            v = v+1
print(v/s)


0.019148936170212766


In [12]:
# 0.02147239263803681
# R1 / G /  
# 8.25 s ± 389 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
# 

In [33]:
%%timeit -n 1
s = len(users)*len(gs)
v = 0
for u in users:
    for g in gs:
        if Test_relation1.objects.filter(staff_member = u, gallery = g).exists():
            v = v+1
        
print(v/s)


0.02875
0.02875
0.02875
0.02875
0.02875
0.02875
0.02875
824 ms ± 24.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
# 0.02147239263803681
# GU / G / 
# 10.3 s ± 493 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

In [34]:
%%timeit -n 1
s = len(users)*len(gs)
v = 0
for u in users:
    for g in gs:
        if GU_Relation.objects.filter(staff_member = u, gallery = g, warehouse__isnull = True).exists():
            v = v+1

print(v/s)

0.02875
0.02875
0.02875
0.02875
0.02875
0.02875
0.02875
941 ms ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
